### Generate 3D Download Links Only

In [ ]:
import pandas as pd
import re

# Load your input Excel file
df = pd.read_excel(r"D:\TMH\Proton_Lab\Gokul_Project\Boswellia Serrata Phytochemicals.xlsx")  # Replace with your actual filename

# Store hyperlinks
hyperlinks = []

# Loop through each row
for index, row in df.iterrows():
    pubchem_id = str(row["PubChem ID"]).strip()

    # Extract numeric CID
    match = re.search(r"CID:(\d+)", pubchem_id)
    if match:
        cid = match.group(1)
        # Construct 3D structure download link
        url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/CID/{cid}/record/SDF?record_type=3d&response_type=save&response_basename=Conformer3D_COMPOUND_CID_{cid}"
        # Excel-friendly hyperlink formula
        hyperlink = f'=HYPERLINK("{url}", "Download")'
    else:
        hyperlink = "NotFound"

    hyperlinks.append(hyperlink)

# Add new column to DataFrame
df["3D Structure Link"] = hyperlinks

# Save to Excel (.xlsx) with formulas preserved
with pd.ExcelWriter(r"D:\TMH\Proton_Lab\Gokul_Project\imppat_with_3D_links.xlsx", engine="openpyxl") as writer:
    df.to_excel(writer, index=False)

print("Excel file saved as 'imppat_with_3D_links.xlsx'")
